# Notebook 10: RAG Evaluation with Ragas Framework

**Module 3: Traditional RAG Implementation**

---

## 📚 What You'll Learn

This notebook covers the complete lifecycle of building and evaluating a production-quality RAG system:

1. ✅ **Basic RAG Pipeline** - Document loading, chunking, embeddings, retrieval, generation
2. ✅ **Building RAG Applications** - Prompt engineering, response formatting, handling edge cases
3. ✅ **Evaluation and Testing** - Comprehensive evaluation with Ragas framework

### Why Evaluation Matters

Building a RAG system is only half the battle. Without proper evaluation, you can't:
- Know if your system is producing accurate answers
- Compare different configuration choices
- Identify areas for improvement
- Ensure production-ready quality

**This notebook introduces the Ragas evaluation framework** - the industry-standard tool for measuring RAG system performance.

---

## 🎯 Learning Objectives

By the end of this notebook, you will:

- Understand the importance of RAG evaluation
- Implement a complete RAG pipeline from scratch
- Integrate the Ragas evaluation framework
- Create comprehensive test datasets with ground truth
- **Evaluate RAG systems across all 6 key non-multimodal metrics**
- Compare multiple RAG configurations systematically
- Visualize and interpret evaluation results
- Apply production best practices

---

## 📖 Table of Contents

1. **Introduction & Setup** (🔰 BEGINNER)
2. **Custom Dummy Data Creation** (🔰/🎓 BEGINNER/INTERMEDIATE)
3. **Basic RAG Pipeline Implementation** (🎓 INTERMEDIATE)
4. **Introduction to Ragas Framework** (🔰 BEGINNER)
5. **Creating Evaluation Dataset** (🎓 INTERMEDIATE)
6. **Ragas Metrics Implementation - All 6 Metrics** (🎓/🚀 INTERMEDIATE/ADVANCED)
7. **Running Comprehensive Evaluation** (🎓 INTERMEDIATE)
8. **Advanced Evaluation - Multiple Configurations** (🚀 ADVANCED)
9. **Results Analysis & Visualization** (🎓/🚀 INTERMEDIATE/ADVANCED)
10. **Best Practices & Summary** (🔰/🎓 BEGINNER/INTERMEDIATE)

---

**Difficulty Level:** 🎓 Intermediate (assumes familiarity with RAG basics from Notebooks 01-07)

**Estimated Time:** 90-120 minutes

**Prerequisites:**
- Completed Notebooks 01-07 (or equivalent RAG knowledge)
- OpenAI API key
- Python 3.9+

Let's begin! 🚀

---

# Section 1: Introduction & Setup

**Difficulty:** 🔰 BEGINNER

In this section, we'll:
1. Install the Ragas evaluation framework
2. Import all required libraries
3. Load environment variables
4. Verify our setup

## 1.1 Install Ragas and Dependencies

Ragas (Retrieval Augmented Generation Assessment) is the industry-standard framework for evaluating RAG systems.

**New packages we're installing:**
- `ragas==0.3.9` - RAG evaluation framework (latest stable version)
- `datasets>=2.14.0` - Required by Ragas for dataset handling
- `seaborn>=0.12.0` - Enhanced visualizations

In [ ]:
# Install Ragas and required dependencies
# Note: Uncomment the lines below if you haven't installed these packages yet

# !pip install ragas==0.3.7 datasets>=2.14.0 seaborn>=0.12.0 -q
# !pip install opik --quiet  # OPIK for experiment tracking and observability

## 1.2 Import Required Libraries

We'll import all necessary libraries for our RAG pipeline and evaluation.

In [1]:
# Standard library imports
import os
import warnings
from pathlib import Path
from datetime import datetime

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Environment variables
from dotenv import load_dotenv

# LangChain core imports (LCEL patterns)
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# LangChain components
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS

# Ragas evaluation framework (updated for 0.3.7)
from ragas import evaluate, EvaluationDataset, SingleTurnSample
from ragas.metrics import (
    Faithfulness,
    AnswerRelevancy,
    ContextPrecision,
    ContextRecall,
    ContextEntityRecall,
    NoiseSensitivity,
)
# Ragas LangChain integration wrappers (required for 0.3.9)
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

# HuggingFace datasets for Ragas
from datasets import Dataset

# OPIK for experiment tracking and observability
try:
    import opik
    from opik import track, Opik
    OPIK_AVAILABLE = True
    print("✓ OPIK available for experiment tracking")
except ImportError:
    OPIK_AVAILABLE = False
    print("ℹ OPIK not installed. Install with: pip install opik")

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

# Set visualization style
plt.style.use('default')
sns.set_palette("husl")

print("✓ All imports successful!")
print("✓ Using LangChain 1.0+ with LCEL patterns")
print("✓ Ragas 0.3.7 evaluation framework loaded")

✓ OPIK available for experiment tracking
✓ All imports successful!
✓ Using LangChain 1.0+ with LCEL patterns
✓ Ragas 0.3.7 evaluation framework loaded


## 1.3 Load Environment Variables

Load your OpenAI API key from the `.env` file.

In [2]:
# Load environment variables from .env file
load_dotenv()

# Verify OpenAI API key is loaded
if not os.getenv("OPENAI_API_KEY"):
    print("❌ WARNING: OPENAI_API_KEY not found!")
    print("Please set it in .env file or export it as an environment variable")
else:
    print(f"✓ OpenAI API Key loaded: {os.getenv('OPENAI_API_KEY')[:8]}...")
    print("✓ Environment setup complete!")

✓ OpenAI API Key loaded: sk-proj-...
✓ Environment setup complete!


## 1.4 Configure OPIK (Optional)

OPIK provides experiment tracking and observability for your RAG evaluations.

**Setup Steps:**
1. Sign up at https://www.comet.com/signup (free tier available)
2. Get your API key from https://www.comet.com/opik/settings
3. Set `OPIK_API_KEY` in your `.env` file or run `opik configure` in terminal
4. OPIK will automatically track all evaluation runs

In [ ]:
#!opik configure

Which Opik deployment do you want to log your traces to?
) - Opik Cloud (default 
2 - Self-hosted Comet platform
3 - Local deployment

> 
Aborted!


In [3]:
# Configure OPIK for experiment tracking
if OPIK_AVAILABLE:
    try:
        # Initialize OPIK client
        opik_client = Opik()
        
        # Create a project for this notebook
        project_name = "RAG_Evaluation_Ragas"
        
        print("✓ OPIK configured successfully!")
        print(f"  Project: {project_name}")
        print(f"  Dashboard: https://www.comet.com/opik/projects/{project_name}")
        print("\n📊 All evaluation results will be logged to OPIK")
        
    except Exception as e:
        print(f"⚠️ OPIK configuration failed: {e}")
        print("  Continuing without OPIK tracking...")
        OPIK_AVAILABLE = False
else:
    print("ℹ OPIK not available - skipping experiment tracking")
    print("  To enable: pip install opik && opik configure")

✓ OPIK configured successfully!
  Project: RAG_Evaluation_Ragas
  Dashboard: https://www.comet.com/opik/projects/RAG_Evaluation_Ragas

📊 All evaluation results will be logged to OPIK


---

# Section 2: Custom Dummy Data Creation

**Difficulty:** 🔰/🎓 BEGINNER/INTERMEDIATE

We'll create a comprehensive set of documentation for a fictional cloud platform called **CloudFlow**.

## Why CloudFlow Platform?

- Technical enough for meaningful Q&A evaluation
- Clear documentation structure supports various question types
- Realistic enterprise scenario
- Includes specific facts and numbers for testing accuracy

## Document Categories (18 documents total)

1. **Architecture** (3 docs) - System design, components, scaling
2. **API Documentation** (4 docs) - Authentication, endpoints, rate limits, errors
3. **Security** (2 docs) - Features and compliance
4. **Pricing** (2 docs) - Tiers and billing
5. **Best Practices** (3 docs) - Optimization, monitoring, DR
6. **Troubleshooting** (3 docs) - Common errors and solutions

In [4]:
# Create CloudFlow Platform Documentation
# Each document is 200-400 words with specific facts for evaluation

cloudflow_docs = [
    # ============================================================================
    # ARCHITECTURE DOCUMENTS (3)
    # ============================================================================
    Document(
        page_content="""CloudFlow Architecture Overview

CloudFlow is a distributed cloud platform built on microservices architecture. The platform consists of three main layers that work together to provide a robust, scalable infrastructure.

The API Gateway layer handles all incoming requests using OAuth 2.0 authentication and routes them through our service mesh powered by Istio. This layer provides load balancing, SSL termination, and request routing capabilities.

The Service Mesh layer orchestrates communication between microservices, providing service discovery, health checking, and automatic failover. It uses Kubernetes for container orchestration across multiple availability zones.

The Data Storage layer implements a distributed database system with automatic replication across three availability zones. This ensures data durability and supports horizontal scaling based on demand.

CloudFlow guarantees 99.99% uptime SLA with triple redundancy across availability zones. The platform supports horizontal scaling with automatic load balancing, allowing each service to scale independently based on CPU and memory metrics.""",
        metadata={"source": "architecture_overview", "topic": "architecture", "difficulty": "intermediate"}
    ),
    
    Document(
        page_content="""CloudFlow Scaling Mechanisms

CloudFlow implements sophisticated auto-scaling mechanisms to handle varying workloads efficiently. The platform monitors real-time metrics to make intelligent scaling decisions.

Horizontal Pod Autoscaling (HPA) adjusts the number of pod replicas based on CPU utilization (target: 70%) and memory usage (target: 80%). When these thresholds are exceeded for more than 3 consecutive minutes, the system automatically provisions additional pods.

Vertical scaling adjusts resource allocation for individual services. CloudFlow can increase or decrease CPU and memory limits without downtime, using Kubernetes resource management capabilities.

The platform supports bursting to handle sudden traffic spikes. During burst periods, CloudFlow can temporarily scale up to 500% of baseline capacity for up to 15 minutes before triggering permanent scaling.

Load balancing distributes traffic across all available pods using a weighted round-robin algorithm. Health checks run every 10 seconds, and unhealthy pods are automatically removed from the rotation within 30 seconds.""",
        metadata={"source": "scaling_guide", "topic": "architecture", "difficulty": "advanced"}
    ),
    
    Document(
        page_content="""CloudFlow System Components

CloudFlow's architecture comprises several key components that work in harmony to deliver reliable cloud services.

The Control Plane manages the overall system state, including service registration, configuration management, and orchestration. It runs on a dedicated cluster with five replicas for high availability.

The Data Plane handles actual request processing and data flow. It consists of worker nodes that execute application workloads and process user requests. Each data plane node has 16 CPU cores and 64GB RAM.

The Observability Stack includes Prometheus for metrics collection, Grafana for visualization, and ELK (Elasticsearch, Logstash, Kibana) for log aggregation. Metrics are collected every 15 seconds and retained for 90 days.

The Service Registry maintains a real-time directory of all available services and their endpoints. It uses etcd for distributed consensus and supports automatic service discovery with DNS-based lookups.

The Message Queue system, based on Apache Kafka, handles asynchronous communication between services with guaranteed message delivery and ordering.""",
        metadata={"source": "system_components", "topic": "architecture", "difficulty": "intermediate"}
    ),
    
    # ============================================================================
    # API DOCUMENTATION (4)
    # ============================================================================
    Document(
        page_content="""CloudFlow API Authentication

CloudFlow APIs support two authentication methods: OAuth 2.0 and API Keys. Both methods provide secure access to platform resources.

OAuth 2.0 is recommended for user-facing applications. It supports the Authorization Code flow and provides access tokens valid for 1 hour and refresh tokens valid for 30 days. To implement OAuth 2.0, direct users to the authorization endpoint at https://auth.cloudflow.io/oauth/authorize with your client_id and redirect_uri parameters.

API Keys are ideal for server-to-server communication and background jobs. Each API key has the format "cf_live_" followed by 32 alphanumeric characters. API keys never expire unless explicitly revoked.

To authenticate requests, include your API key in the Authorization header: "Authorization: Bearer YOUR_API_KEY". All API requests must be made over HTTPS; HTTP requests will be rejected with a 403 error.

API keys can be scoped to specific permissions (read, write, admin) and restricted to specific IP addresses for enhanced security. You can manage your API keys through the CloudFlow dashboard or the /api/v1/keys endpoint.""",
        metadata={"source": "api_authentication", "topic": "api", "difficulty": "beginner"}
    ),
    
    Document(
        page_content="""CloudFlow REST API Endpoints

CloudFlow provides a comprehensive REST API with endpoints organized by resource type. All endpoints follow RESTful conventions and return JSON responses.

Base URL: https://api.cloudflow.io/v1

Resources endpoint: GET /api/v1/resources - List all resources with pagination (max 100 per page). Supports filtering by type, status, and creation date.

Resource creation: POST /api/v1/resources - Create a new resource. Required fields: name (string), type (string), config (object). Returns 201 Created on success.

Resource details: GET /api/v1/resources/{id} - Retrieve detailed information about a specific resource by ID.

Resource update: PUT /api/v1/resources/{id} - Update an existing resource. Supports partial updates with PATCH /api/v1/resources/{id}.

Resource deletion: DELETE /api/v1/resources/{id} - Delete a resource. Returns 204 No Content on success. Deleted resources are soft-deleted and can be recovered within 30 days.

All list endpoints support query parameters: limit (default: 25, max: 100), offset (default: 0), sort (default: created_at), order (asc|desc).""",
        metadata={"source": "api_endpoints", "topic": "api", "difficulty": "intermediate"}
    ),
    
    Document(
        page_content="""CloudFlow API Rate Limiting

CloudFlow implements rate limiting to ensure fair usage and platform stability. Rate limits vary by pricing tier and authentication method.

Standard Tier: 1,000 requests per hour per API key. Burst capacity allows up to 100 requests per minute. Exceeding limits returns HTTP 429 (Too Many Requests).

Premium Tier: 10,000 requests per hour per API key with burst capacity of 500 requests per minute. Premium tier also includes priority request processing.

Enterprise Tier: Custom rate limits negotiated based on usage patterns. Typically starts at 100,000 requests per hour with dedicated infrastructure.

Rate limit headers are included in every response:
- X-RateLimit-Limit: Maximum requests per hour
- X-RateLimit-Remaining: Remaining requests in current window
- X-RateLimit-Reset: Unix timestamp when the limit resets

When rate limited, the Retry-After header indicates how many seconds to wait before retrying. Implement exponential backoff: wait 1s, then 2s, then 4s, etc.

OAuth 2.0 authenticated requests have separate, higher limits: 5,000 requests per hour for Standard tier.""",
        metadata={"source": "api_rate_limits", "topic": "api", "difficulty": "intermediate"}
    ),
    
    Document(
        page_content="""CloudFlow API Error Codes

CloudFlow APIs use standard HTTP status codes and provide detailed error messages in JSON format to help diagnose issues.

Authentication Errors:
- 401 Unauthorized: Missing or invalid API key. Check the Authorization header.
- 403 Forbidden: Valid API key but insufficient permissions for the requested operation.

Client Errors:
- 400 Bad Request: Invalid request format or missing required fields. The response includes a "details" field explaining what's wrong.
- 404 Not Found: Requested resource doesn't exist. Verify the resource ID.
- 409 Conflict: Request conflicts with current resource state (e.g., duplicate name).
- 422 Unprocessable Entity: Request format is valid but contains semantic errors.
- 429 Too Many Requests: Rate limit exceeded. Check X-RateLimit-Reset header.

Server Errors:
- 500 Internal Server Error: Unexpected server error. CloudFlow team is automatically notified.
- 502 Bad Gateway: Temporary issue with upstream services. Retry after a few seconds.
- 503 Service Unavailable: Scheduled maintenance or system overload. Check status.cloudflow.io.

Error Response Format: {"error": {"code": "error_code", "message": "Human-readable message", "details": {...}}}""",
        metadata={"source": "api_error_codes", "topic": "api", "difficulty": "beginner"}
    ),
    
    # ============================================================================
    # SECURITY DOCUMENTATION (2)
    # ============================================================================
    Document(
        page_content="""CloudFlow Security Features

Security is a top priority at CloudFlow. We implement industry-leading security practices to protect your data and applications.

Encryption: All data is encrypted at rest using AES-256 encryption. Data in transit uses TLS 1.3 with perfect forward secrecy. Encryption keys are rotated every 90 days using AWS KMS.

Network Security: CloudFlow runs in a Virtual Private Cloud (VPC) with strict network segmentation. Public endpoints are protected by Web Application Firewall (WAF) rules that block common attack patterns. DDoS protection is provided by Cloudflare with mitigation capacity up to 50 Gbps.

Access Control: All resources support Role-Based Access Control (RBAC) with customizable roles and permissions. We support integration with external identity providers via SAML 2.0 and OpenID Connect.

Audit Logging: Every API call is logged with timestamp, user identity, IP address, and action taken. Audit logs are immutable and retained for 2 years. You can access logs via the /api/v1/audit-logs endpoint.

Vulnerability Management: CloudFlow undergoes quarterly penetration testing by independent security firms. We maintain a bug bounty program and respond to security reports within 24 hours.""",
        metadata={"source": "security_features", "topic": "security", "difficulty": "intermediate"}
    ),
    
    Document(
        page_content="""CloudFlow Compliance Standards

CloudFlow maintains compliance with major industry standards and regulations to ensure your data is handled responsibly.

SOC 2 Type II: CloudFlow is SOC 2 Type II certified, demonstrating our commitment to security, availability, and confidentiality. Audit reports are available to enterprise customers under NDA.

GDPR Compliance: CloudFlow is fully compliant with the European Union's General Data Protection Regulation. We support data residency requirements, right to erasure, data portability, and provide Data Processing Agreements (DPA) to all customers.

HIPAA: For healthcare customers, CloudFlow offers HIPAA-compliant infrastructure with Business Associate Agreements (BAA). HIPAA features include enhanced audit logging, encrypted backups, and strict access controls.

ISO 27001: CloudFlow's information security management system is certified to ISO 27001:2013 standards. We maintain comprehensive security policies and undergo annual recertification audits.

PCI DSS: For customers processing payment card data, CloudFlow provides PCI DSS Level 1 certified infrastructure. However, we recommend using dedicated payment processors rather than storing card data.

Data Residency: CloudFlow supports data residency in US, EU, UK, and APAC regions to meet local regulatory requirements.""",
        metadata={"source": "compliance_standards", "topic": "security", "difficulty": "advanced"}
    ),
    
    # ============================================================================
    # PRICING DOCUMENTATION (2)
    # ============================================================================
    Document(
        page_content="""CloudFlow Pricing Tiers

CloudFlow offers three pricing tiers designed to meet the needs of individuals, teams, and enterprises.

Standard Tier ($99/month):
- 1,000 API requests per hour
- 100 GB storage included
- 10 GB bandwidth per month
- Community support via forums
- 99.9% uptime SLA
- Up to 5 team members

Premium Tier ($499/month):
- 10,000 API requests per hour
- 1 TB storage included
- 100 GB bandwidth per month
- Email support with 24-hour response time
- 99.95% uptime SLA
- Up to 25 team members
- Advanced monitoring and alerting
- Custom domain support

Enterprise Tier (Custom pricing):
- Custom API rate limits (100,000+ requests/hour)
- Unlimited storage and bandwidth
- 24/7 phone and email support with 1-hour response time
- 99.99% uptime SLA with service credits
- Unlimited team members
- Dedicated account manager
- Custom integrations and professional services
- Private cloud deployment options

All tiers include: SSL certificates, daily backups, API access, and dashboard analytics. Annual billing provides 15% discount.""",
        metadata={"source": "pricing_tiers", "topic": "pricing", "difficulty": "beginner"}
    ),
    
    Document(
        page_content="""CloudFlow Billing Information

Understanding CloudFlow's billing model helps you manage costs effectively and avoid unexpected charges.

Billing Cycle: Subscriptions are billed monthly on the date you signed up. Annual subscriptions are billed upfront with a 15% discount. Billing date can be changed once per year.

Usage-Based Charges: Beyond included quotas, additional usage is billed at:
- API requests: $0.01 per 1,000 requests
- Storage: $0.10 per GB per month
- Bandwidth: $0.08 per GB
- Backup retention (beyond 30 days): $0.05 per GB per month

Payment Methods: CloudFlow accepts credit cards (Visa, Mastercard, Amex), ACH transfers (US only), and wire transfers for invoices over $1,000. Cryptocurrency payments available for annual plans.

Invoicing: Invoices are emailed on the billing date and available in the dashboard. Enterprise customers receive consolidated monthly invoices with 30-day payment terms.

Upgrades and Downgrades: Upgrade anytime to immediately access higher tier features. Downgrades take effect at the next billing cycle. Prorated credits are applied to your account balance.

Free Trial: New customers get 14-day free trial on Premium tier with no credit card required. Trial includes 1,000 API requests and 10 GB storage.""",
        metadata={"source": "billing_info", "topic": "pricing", "difficulty": "beginner"}
    ),
    
    # ============================================================================
    # BEST PRACTICES DOCUMENTATION (3)
    # ============================================================================
    Document(
        page_content="""CloudFlow Performance Optimization

Following these best practices will help you achieve optimal performance from your CloudFlow applications.

Caching Strategy: Implement caching at multiple levels. Use CloudFlow's built-in Redis cache for frequently accessed data with TTL between 5-60 minutes. Cache API responses on the client side and respect Cache-Control headers.

Request Optimization: Batch multiple operations into single API calls when possible. Use pagination for large result sets (recommended page size: 50-100 items). Implement request compression using gzip to reduce bandwidth.

Connection Management: Reuse HTTP connections with keep-alive headers. Maintain a connection pool with 5-10 concurrent connections per API key. Set appropriate timeouts: connection timeout 10s, read timeout 30s.

Query Efficiency: Use field filtering to request only required data: /resources?fields=id,name,status. Leverage server-side filtering instead of retrieving all data and filtering locally.

Asynchronous Processing: For long-running operations, use CloudFlow's async API endpoints. Poll for results using the returned job_id rather than blocking on the initial request.

CDN Usage: Serve static assets through CloudFlow's global CDN with 150+ edge locations. Configure appropriate cache headers for optimal performance: max-age=3600 for semi-static content.""",
        metadata={"source": "performance_optimization", "topic": "best_practices", "difficulty": "intermediate"}
    ),
    
    Document(
        page_content="""CloudFlow Monitoring and Observability

Effective monitoring ensures your CloudFlow applications remain healthy and performant.

Metrics Collection: CloudFlow automatically collects key metrics including request rate, error rate, latency (p50, p95, p99), and resource utilization. Access metrics via the dashboard or Metrics API at /api/v1/metrics.

Custom Metrics: Send custom application metrics using the StatsD protocol. CloudFlow aggregates custom metrics every 60 seconds and retains them for 90 days.

Alerting: Configure alerts for critical conditions like error rate >5%, latency >500ms, or approaching rate limits. CloudFlow supports alerting via email, SMS, Slack, PagerDuty, and webhooks.

Distributed Tracing: Enable distributed tracing to track requests across services. CloudFlow supports OpenTelemetry and provides trace visualization in the dashboard. Sample rate: 10% of requests (configurable up to 100%).

Log Management: CloudFlow retains logs for 7 days by default (30 days for Premium, 90 days for Enterprise). Use structured logging with JSON format for better searchability. Maximum log line length: 32KB.

Dashboard Widgets: Create custom dashboards with real-time metrics, SLA compliance, and cost tracking. Share dashboards with team members or embed in external tools using iframe integration.""",
        metadata={"source": "monitoring_observability", "topic": "best_practices", "difficulty": "intermediate"}
    ),
    
    Document(
        page_content="""CloudFlow Disaster Recovery

CloudFlow implements comprehensive disaster recovery capabilities to protect your data and ensure business continuity.

Backup Strategy: CloudFlow performs automatic daily backups of all data at 2 AM UTC. Backups are encrypted and stored in geographically diverse locations. Retention: 30 days for Standard tier, 90 days for Premium, 1 year for Enterprise.

Point-in-Time Recovery: Enterprise customers can restore data to any point within the retention period with 5-minute granularity. Recovery operations typically complete within 15-30 minutes.

Multi-Region Replication: Enable multi-region replication for critical data. Data is asynchronously replicated to a secondary region within 60 seconds. Failover to secondary region is automatic and takes approximately 5 minutes.

Backup Verification: CloudFlow performs monthly backup restoration tests to ensure data recoverability. Test results are available in your compliance dashboard.

Export Capabilities: Export your data anytime in JSON, CSV, or Parquet format. Full exports are available via the /api/v1/export endpoint. Large exports (>10 GB) are delivered to your S3 bucket.

RTO and RPO: CloudFlow guarantees Recovery Time Objective (RTO) of 4 hours and Recovery Point Objective (RPO) of 1 hour for Enterprise tier. Contact support to initiate disaster recovery procedures.""",
        metadata={"source": "disaster_recovery", "topic": "best_practices", "difficulty": "advanced"}
    ),
    
    # ============================================================================
    # TROUBLESHOOTING DOCUMENTATION (3)
    # ============================================================================
    Document(
        page_content="""Common CloudFlow Errors and Solutions

This guide covers the most common errors encountered when using CloudFlow and their solutions.

Error: "Invalid API Key" (401)
Solution: Verify your API key format starts with "cf_live_" and is exactly 40 characters. Check for extra spaces or newlines. Generate a new API key if the issue persists. API keys are case-sensitive.

Error: "Rate Limit Exceeded" (429)
Solution: Implement exponential backoff in your retry logic. Check X-RateLimit-Reset header to know when limits reset. Consider upgrading to a higher tier if you consistently hit limits. Use batch endpoints to reduce request count.

Error: "Resource Not Found" (404)
Solution: Verify the resource ID is correct and the resource hasn't been deleted. Use the /api/v1/resources endpoint to list available resources. Check if you're using the correct API version (/v1).

Error: "Timeout" (504)
Solution: Increase client timeout to at least 30 seconds. For long-running operations, use async endpoints and poll for results. Check CloudFlow status page for any service degradation.

Error: "Validation Error" (422)
Solution: Review the error details field for specific validation failures. Common issues: missing required fields, invalid data types, values outside allowed ranges. Consult API documentation for correct request format.""",
        metadata={"source": "common_errors", "topic": "troubleshooting", "difficulty": "beginner"}
    ),
    
    Document(
        page_content="""CloudFlow Debugging Guide

When troubleshooting issues with CloudFlow, follow this systematic debugging approach.

Step 1 - Check Service Status: Visit status.cloudflow.io to verify all systems are operational. Subscribe to status updates to receive notifications about incidents and maintenance.

Step 2 - Review API Logs: Access detailed API logs in the CloudFlow dashboard under Analytics > API Logs. Filter by time range, status code, and endpoint. Look for patterns in failed requests.

Step 3 - Enable Debug Mode: Add X-CloudFlow-Debug: true header to requests to receive detailed debug information in responses. Debug mode provides request ID, processing time breakdown, and backend service information.

Step 4 - Test with curl: Isolate issues by testing with curl commands. Example: curl -H "Authorization: Bearer YOUR_API_KEY" -H "X-CloudFlow-Debug: true" https://api.cloudflow.io/v1/resources

Step 5 - Check Network Connectivity: Ensure your network allows outbound HTTPS traffic to *.cloudflow.io on port 443. Verify DNS resolution is working correctly.

Step 6 - Verify SDK Version: If using CloudFlow SDK, ensure you're running the latest version. Outdated SDKs may not support new API features or may have known bugs.

Step 7 - Contact Support: If issues persist, contact CloudFlow support with the request ID from failed requests. Support responds within 24 hours for Standard tier, 4 hours for Premium, 1 hour for Enterprise.""",
        metadata={"source": "debugging_guide", "topic": "troubleshooting", "difficulty": "intermediate"}
    ),
    
    Document(
        page_content="""CloudFlow Support Escalation Process

Understanding CloudFlow's support escalation process ensures your issues are resolved efficiently.

Support Channels:
- Community Forums (All tiers): community.cloudflow.io - Best for general questions, feature requests, and sharing knowledge
- Email Support (Premium & Enterprise): support@cloudflow.io - Include account ID and request ID in subject line
- Phone Support (Enterprise only): +1-888-CLOUDFLOW - Available 24/7 for critical issues
- Slack Channel (Enterprise only): Direct access to engineering team

Issue Severity Levels:
- P0 (Critical): Complete service outage affecting production. Response time: 1 hour for Enterprise, 4 hours for Premium
- P1 (High): Major functionality impaired but workarounds available. Response time: 4 hours for Enterprise, 8 hours for Premium
- P2 (Medium): Minor functionality issues with workarounds. Response time: 24 hours
- P3 (Low): Questions, feature requests, documentation issues. Response time: 48 hours

Escalation Path: If your issue isn't resolved within SLA, it automatically escalates to the next support tier. Enterprise customers can request immediate escalation to engineering team.

Required Information: Include account ID, request ID, error messages, timestamps, steps to reproduce, and expected vs actual behavior. Screenshots and API logs are helpful.""",
        metadata={"source": "support_escalation", "topic": "troubleshooting", "difficulty": "beginner"}
    ),
]

print(f"✓ Created {len(cloudflow_docs)} CloudFlow documentation documents")
print("\nDocument breakdown by category:")
for topic in ["architecture", "api", "security", "pricing", "best_practices", "troubleshooting"]:
    count = len([doc for doc in cloudflow_docs if doc.metadata["topic"] == topic])
    print(f"  - {topic.title()}: {count} documents")

✓ Created 17 CloudFlow documentation documents

Document breakdown by category:
  - Architecture: 3 documents
  - Api: 4 documents
  - Security: 2 documents
  - Pricing: 2 documents
  - Best_Practices: 3 documents
  - Troubleshooting: 3 documents


## 2.2 Preview Sample Documents

Let's look at a sample document to understand the structure:

In [5]:
# Preview first document
sample_doc = cloudflow_docs[0]

print("Sample Document:")
print("=" * 80)
print(f"Content preview (first 300 chars):\n{sample_doc.page_content[:300]}...\n")
print(f"Metadata: {sample_doc.metadata}")
print("=" * 80)

# Calculate total characters
total_chars = sum(len(doc.page_content) for doc in cloudflow_docs)
avg_chars = total_chars / len(cloudflow_docs)

print(f"\nDataset statistics:")
print(f"  Total documents: {len(cloudflow_docs)}")
print(f"  Total characters: {total_chars:,}")
print(f"  Average document length: {int(avg_chars)} characters")

Sample Document:
Content preview (first 300 chars):
CloudFlow Architecture Overview

CloudFlow is a distributed cloud platform built on microservices architecture. The platform consists of three main layers that work together to provide a robust, scalable infrastructure.

The API Gateway layer handles all incoming requests using OAuth 2.0 authenticat...

Metadata: {'source': 'architecture_overview', 'topic': 'architecture', 'difficulty': 'intermediate'}

Dataset statistics:
  Total documents: 17
  Total characters: 20,987
  Average document length: 1234 characters


---

# Section 3: Basic RAG Pipeline Implementation

**Difficulty:** 🎓 INTERMEDIATE

Now we'll implement a complete RAG pipeline following LangChain 1.0+ best practices. This includes:

1. Text splitting with optimal chunking
2. Embeddings generation
3. Vector store creation and persistence
4. Retriever configuration
5. LLM setup
6. Prompt engineering
7. LCEL chain construction

## 3.1 Text Splitting

We'll use `RecursiveCharacterTextSplitter` - the recommended splitter for most use cases.

In [6]:
# Initialize text splitter with optimal parameters
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1024,           # ~200-250 tokens, optimal for most Q&A tasks
    chunk_overlap=128,         # 12.5% overlap maintains context continuity
    separators=["\n\n", "\n", " ", ""]  # Prioritize paragraph > line > word boundaries
)

# Split all CloudFlow documents into chunks
chunks = text_splitter.split_documents(cloudflow_docs)

print(f"✓ Split {len(cloudflow_docs)} documents into {len(chunks)} chunks")
print(f"\nChunk statistics:")
chunk_lengths = [len(chunk.page_content) for chunk in chunks]
print(f"  Average chunk size: {int(np.mean(chunk_lengths))} characters")
print(f"  Min chunk size: {min(chunk_lengths)} characters")
print(f"  Max chunk size: {max(chunk_lengths)} characters")

# Preview a chunk
print(f"\nSample chunk:")
print(f"Content: {chunks[0].page_content[:200]}...")
print(f"Metadata: {chunks[0].metadata}")

✓ Split 17 documents into 34 chunks

Chunk statistics:
  Average chunk size: 616 characters
  Min chunk size: 105 characters
  Max chunk size: 1012 characters

Sample chunk:
Content: CloudFlow Architecture Overview

CloudFlow is a distributed cloud platform built on microservices architecture. The platform consists of three main layers that work together to provide a robust, scala...
Metadata: {'source': 'architecture_overview', 'topic': 'architecture', 'difficulty': 'intermediate'}


## 3.2 Create Embeddings and Vector Store

We'll use OpenAI's `text-embedding-3-small` model - cost-effective and performant.

In [7]:
# Initialize embeddings model
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",  # 1536 dimensions
    # Cost: $0.00002 per 1K tokens
)

print("Creating FAISS vector store...")
print(f"  Embedding {len(chunks)} chunks...")

# Create FAISS vector store from chunks
# This may take 30-60 seconds depending on number of chunks
vectorstore = FAISS.from_documents(chunks, embeddings)

# Save vector store to disk for reusability
vectorstore_path = "./ragas_evaluation_faiss"
vectorstore.save_local(vectorstore_path)

print(f"✓ FAISS vector store created and saved to '{vectorstore_path}'")
print(f"✓ Indexed {len(chunks)} document chunks")
print("\nVector store can be reloaded with:")
print(f"  vectorstore = FAISS.load_local('{vectorstore_path}', embeddings, allow_dangerous_deserialization=True)")

Creating FAISS vector store...
  Embedding 34 chunks...
✓ FAISS vector store created and saved to './ragas_evaluation_faiss'
✓ Indexed 34 document chunks

Vector store can be reloaded with:
  vectorstore = FAISS.load_local('./ragas_evaluation_faiss', embeddings, allow_dangerous_deserialization=True)


## 3.3 Create Retriever

Configure the retriever with similarity search and k=4 (returns top 4 most relevant chunks).

In [8]:
# Create retriever from vector store
retriever = vectorstore.as_retriever(
    search_type="similarity",  # Simple similarity search (vs MMR for diversity)
    search_kwargs={"k": 4}     # Retrieve top 4 most relevant chunks
)

print("✓ Retriever configured successfully")
print("  Search type: similarity")
print("  Number of results (k): 4")

# Test retriever with a sample query
test_query = "What is CloudFlow's uptime SLA?"
print(f"\nTesting retriever with query: '{test_query}'")

retrieved_docs = retriever.invoke(test_query)
print(f"✓ Retrieved {len(retrieved_docs)} documents")
print(f"\nFirst retrieved chunk preview:")
print(f"{retrieved_docs[0].page_content[:250]}...")

✓ Retriever configured successfully
  Search type: similarity
  Number of results (k): 4

Testing retriever with query: 'What is CloudFlow's uptime SLA?'
✓ Retrieved 4 documents

First retrieved chunk preview:
CloudFlow guarantees 99.99% uptime SLA with triple redundancy across availability zones. The platform supports horizontal scaling with automatic load balancing, allowing each service to scale independently based on CPU and memory metrics....


## 3.4 Initialize Language Model

We'll use GPT-4o-mini for cost-effective, high-quality responses.

In [9]:
# Initialize ChatOpenAI with GPT-4o-mini
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,        # Deterministic responses for consistent evaluation
    max_tokens=500       # Reasonable limit for Q&A responses
)

print("✓ Language model configured")
print("  Model: gpt-4o-mini")
print("  Temperature: 0 (deterministic)")
print("  Max tokens: 500")

# Quick test
test_response = llm.invoke("Say 'LLM is ready!'")
print(f"\nLLM test: {test_response.content}")

✓ Language model configured
  Model: gpt-4o-mini
  Temperature: 0 (deterministic)
  Max tokens: 500

LLM test: LLM is ready!


## 3.5 Create Prompt Template

Design a prompt that instructs the LLM to answer based on context and handle cases where information is missing.

In [10]:
# Define prompt template for CloudFlow assistant
template = """You are a helpful assistant for CloudFlow Platform documentation.
Answer the question based on the following context. If you cannot answer based on
the context, say "I don't have enough information to answer that question."

Be concise and accurate. Include specific details like numbers, limits, and technical
specifications when available in the context.

Context:
{context}

Question: {question}

Answer:"""

# Create ChatPromptTemplate
prompt = ChatPromptTemplate.from_template(template)

print("✓ Prompt template created")
print("\nPrompt structure:")
print("  1. System instruction (CloudFlow assistant role)")
print("  2. Context from retrieved documents")
print("  3. User question")
print("  4. Answer placeholder")

✓ Prompt template created

Prompt structure:
  1. System instruction (CloudFlow assistant role)
  2. Context from retrieved documents
  3. User question
  4. Answer placeholder


## 3.6 Build RAG Chain with LCEL

Now we'll combine all components using LangChain Expression Language (LCEL) - the modern way to build chains.

In [11]:
# Helper function to format retrieved documents
def format_docs(docs):
    """Combine retrieved document chunks into a single context string."""
    return "\n\n".join(doc.page_content for doc in docs)

# Build RAG chain using LCEL (pipe operator)
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print("✓ RAG chain created using LCEL")
print("\nChain flow:")
print("  1. Question → Retriever → Get relevant documents")
print("  2. Documents → format_docs → Create context string")
print("  3. Context + Question → Prompt template")
print("  4. Prompt → LLM → Generate answer")
print("  5. LLM output → StrOutputParser → Extract text")

✓ RAG chain created using LCEL

Chain flow:
  1. Question → Retriever → Get relevant documents
  2. Documents → format_docs → Create context string
  3. Context + Question → Prompt template
  4. Prompt → LLM → Generate answer
  5. LLM output → StrOutputParser → Extract text


## 3.7 Test the RAG Pipeline

Let's verify our RAG pipeline works correctly before evaluation.

In [12]:
# Test questions to verify RAG pipeline
test_questions = [
    "What is CloudFlow's uptime SLA?",
    "How do I authenticate with CloudFlow APIs?",
    "What are the pricing tiers?"
]

print("Testing RAG pipeline with sample questions:\n")
print("=" * 80)

for i, question in enumerate(test_questions, 1):
    print(f"\nQuestion {i}: {question}")
    answer = rag_chain.invoke(question)
    print(f"Answer: {answer}")
    print("-" * 80)

print("\n✓ RAG pipeline is working correctly!")

Testing RAG pipeline with sample questions:


Question 1: What is CloudFlow's uptime SLA?
Answer: CloudFlow guarantees a 99.99% uptime SLA for the Enterprise Tier, 99.95% for the Premium Tier, and 99.9% for the Standard Tier.
--------------------------------------------------------------------------------

Question 2: How do I authenticate with CloudFlow APIs?
Answer: You can authenticate with CloudFlow APIs using two methods: OAuth 2.0 and API Keys.

1. **OAuth 2.0**: This method is recommended for user-facing applications. To implement it, direct users to the authorization endpoint at `https://auth.cloudflow.io/oauth/authorize` with your `client_id` and `redirect_uri` parameters. Access tokens are valid for 1 hour, and refresh tokens are valid for 30 days.

2. **API Keys**: This method is ideal for server-to-server communication and background jobs. Each API key has the format "cf_live_" followed by 32 alphanumeric characters and never expires unless explicitly revoked. To authentica

---

# Section 4: Introduction to Ragas Framework

**Difficulty:** 🔰 BEGINNER

## What is Ragas?

**Ragas** (Retrieval Augmented Generation Assessment) is the industry-standard framework for evaluating RAG systems. It provides:

- Automated evaluation metrics for RAG quality
- LLM-based and traditional metrics
- Integration with popular frameworks (LangChain, LlamaIndex)
- Comprehensive evaluation across retrieval and generation

## Why Evaluate RAG Systems?

Without evaluation, you can't:
- ❌ Know if answers are accurate and grounded
- ❌ Compare different configurations (chunk size, k value, etc.)
- ❌ Identify failure patterns
- ❌ Optimize for production deployment
- ❌ Track quality over time

## Core Ragas Metrics (6 Non-Multimodal Metrics)

We'll implement all 6 core non-multimodal metrics in this notebook:

### 1. **Faithfulness** 🔍
- **What it measures**: Are the answers grounded in the retrieved context?
- **How it works**: Uses LLM to verify if answer statements can be inferred from context
- **Why it matters**: Prevents hallucinations
- **Target score**: >0.7
- **Speed**: Slow (~2-3s per question, requires LLM calls)

### 2. **Answer Relevancy (Response Relevancy)** 📊
- **What it measures**: Is the answer relevant to the question?
- **How it works**: Uses embeddings to measure semantic relevance
- **Why it matters**: Ensures on-topic responses
- **Target score**: >0.7
- **Speed**: Fast (~0.5s per question)

### 3. **Context Precision** 🎯
- **What it measures**: Are the most relevant chunks ranked highest?
- **How it works**: Checks if relevant chunks appear early in retrieval results
- **Why it matters**: Evaluates retrieval algorithm quality
- **Target score**: >0.6
- **Speed**: Medium (~1s per question)

### 4. **Context Recall** ✅
- **What it measures**: Was all necessary information retrieved?
- **How it works**: Compares retrieved contexts against ground truth
- **Why it matters**: Ensures complete information coverage
- **Target score**: >0.6
- **Speed**: Medium (~1s per question)

### 5. **Context Entity Recall** 🏷️
- **What it measures**: Are key entities from ground truth present in retrieved contexts?
- **How it works**: Extracts entities from ground truth and checks presence in contexts
- **Why it matters**: Ensures important named entities are retrieved
- **Target score**: >0.6
- **Speed**: Medium (~1-2s per question)

### 6. **Noise Sensitivity** 🎚️
- **What it measures**: Is the system robust to irrelevant/noisy context?
- **How it works**: Tests if system can ignore distractors in retrieved contexts
- **Why it matters**: Evaluates robustness in real-world scenarios with imperfect retrieval
- **Target score**: >0.7
- **Speed**: Slow (~2-3s per question)

## Metric Types

**LLM-Based Metrics** (faithfulness, context_precision, context_recall, context_entity_recall, noise_sensitivity):
- More accurate, closer to human evaluation
- Slower and more expensive (requires LLM calls)
- Slightly non-deterministic

**Embedding-Based Metrics** (answer_relevancy):
- Faster and cheaper
- Deterministic
- Good for rapid iteration

**Note**: We're excluding multimodal metrics (Multimodal Faithfulness, Multimodal Relevance) as they're for image/video content evaluation.

## 4.1 Ragas Architecture Overview

Here's how Ragas fits into the evaluation workflow:

```
┌─────────────────┐
│  Test Questions │
│  + Ground Truth │
└────────┬────────┘
         │
         ▼
┌─────────────────┐
│   RAG Pipeline  │ ← Your system
│  (retrieval +   │
│   generation)   │
└────────┬────────┘
         │
         │ Generates: answers + contexts
         ▼
┌─────────────────┐
│ Ragas Evaluator │
│  6 Metrics      │
└────────┬────────┘
         │
         ▼
┌─────────────────┐
│ Evaluation      │
│ Scores          │
└─────────────────┘
```

---

# Section 5: Creating Evaluation Dataset

**Difficulty:** 🎓 INTERMEDIATE

To evaluate our RAG system, we need:
1. **Test questions** - Representative user queries
2. **Ground truth answers** - Reference answers for comparison
3. **Generated answers** - From our RAG pipeline
4. **Retrieved contexts** - The chunks used to generate answers

## 5.1 Create Test Questions with Ground Truth

We'll create 18 diverse questions covering different categories:

In [13]:
# Test cases: question + ground truth answer pairs
test_cases = [
    # ========== SIMPLE FACTUAL (5) ==========
    {
        "question": "What is CloudFlow's uptime SLA?",
        "ground_truth": "CloudFlow guarantees 99.99% uptime SLA with triple redundancy across availability zones."
    },
    {
        "question": "What authentication protocol does CloudFlow use?",
        "ground_truth": "CloudFlow uses OAuth 2.0 for user-facing applications and API keys for server-to-server communication."
    },
    {
        "question": "What is the service mesh technology used by CloudFlow?",
        "ground_truth": "CloudFlow uses Istio as the service mesh technology to orchestrate communication between microservices."
    },
    {
        "question": "What compliance standards does CloudFlow support?",
        "ground_truth": "CloudFlow supports SOC 2 Type II, GDPR, HIPAA, ISO 27001, and PCI DSS Level 1 compliance standards."
    },
    {
        "question": "How long are CloudFlow audit logs retained?",
        "ground_truth": "CloudFlow audit logs are immutable and retained for 2 years."
    },
    
    # ========== MULTI-FACT (4) ==========
    {
        "question": "What are the three main layers of CloudFlow architecture?",
        "ground_truth": "The three main layers are: API Gateway layer (handles authentication and routing), Service Mesh layer (orchestrates microservices), and Data Storage layer (distributed database with replication)."
    },
    {
        "question": "What are CloudFlow's pricing tiers and their API rate limits?",
        "ground_truth": "Standard tier costs $99/month with 1,000 requests/hour, Premium tier costs $499/month with 10,000 requests/hour, and Enterprise tier has custom pricing with 100,000+ requests/hour."
    },
    {
        "question": "What HTTP status codes indicate authentication failures in CloudFlow API?",
        "ground_truth": "401 Unauthorized indicates missing or invalid API key, and 403 Forbidden indicates valid API key but insufficient permissions."
    },
    {
        "question": "What auto-scaling metrics does CloudFlow monitor?",
        "ground_truth": "CloudFlow monitors CPU utilization (target 70%), memory usage (target 80%), and triggers scaling when thresholds are exceeded for more than 3 consecutive minutes."
    },
    
    # ========== PROCEDURAL (3) ==========
    {
        "question": "How do I authenticate with CloudFlow APIs using an API key?",
        "ground_truth": "Include your API key in the Authorization header as 'Authorization: Bearer YOUR_API_KEY'. All requests must be made over HTTPS, and API keys have the format 'cf_live_' followed by 32 alphanumeric characters."
    },
    {
        "question": "How do I handle rate limit errors in CloudFlow?",
        "ground_truth": "When you receive a 429 error, implement exponential backoff in retry logic, check the X-RateLimit-Reset header to know when limits reset, and use the Retry-After header to determine wait time (1s, then 2s, then 4s, etc.)."
    },
    {
        "question": "What steps should I follow to optimize CloudFlow API performance?",
        "ground_truth": "Implement caching with Redis (TTL 5-60 minutes), batch multiple operations into single API calls, use pagination for large result sets (50-100 items), enable request compression with gzip, and maintain a connection pool with 5-10 concurrent connections."
    },
    
    # ========== COMPARISON (2) ==========
    {
        "question": "What's the difference between Standard and Premium tier rate limits?",
        "ground_truth": "Standard tier allows 1,000 requests per hour with 100 requests per minute burst, while Premium tier allows 10,000 requests per hour with 500 requests per minute burst. Premium also includes priority request processing."
    },
    {
        "question": "How does OAuth 2.0 authentication differ from API key authentication in CloudFlow?",
        "ground_truth": "OAuth 2.0 is recommended for user-facing applications with access tokens valid for 1 hour and provides the Authorization Code flow, while API keys are ideal for server-to-server communication, never expire unless revoked, and have a simpler implementation."
    },
    
    # ========== TROUBLESHOOTING (2) ==========
    {
        "question": "What should I do if I receive a 504 timeout error?",
        "ground_truth": "Increase client timeout to at least 30 seconds, use async endpoints for long-running operations and poll for results, and check the CloudFlow status page for any service degradation."
    },
    {
        "question": "How do I debug slow API response times in CloudFlow?",
        "ground_truth": "Add X-CloudFlow-Debug: true header to requests for detailed debug information, review API logs in the dashboard under Analytics > API Logs, test with curl commands, and verify network connectivity to *.cloudflow.io on port 443."
    },
    
    # ========== EDGE CASES (2) ==========
    {
        "question": "What happens if I use an expired OAuth token?",
        "ground_truth": "If you use an expired access token, you'll receive a 401 Unauthorized error. You should use your refresh token to obtain a new access token. Access tokens are valid for 1 hour and refresh tokens are valid for 30 days."
    },
    {
        "question": "Does CloudFlow support blockchain integration?",
        "ground_truth": "I don't have enough information to answer that question."  # Tests 'I don't know' handling
    },
]

print(f"✓ Created {len(test_cases)} test questions with ground truth answers\n")
print("Question breakdown by category:")
print("  - Simple Factual: 5 questions")
print("  - Multi-Fact: 4 questions")
print("  - Procedural: 3 questions")
print("  - Comparison: 2 questions")
print("  - Troubleshooting: 2 questions")
print("  - Edge Cases: 2 questions")

✓ Created 18 test questions with ground truth answers

Question breakdown by category:
  - Simple Factual: 5 questions
  - Multi-Fact: 4 questions
  - Procedural: 3 questions
  - Comparison: 2 questions
  - Troubleshooting: 2 questions
  - Edge Cases: 2 questions


## 5.2 Generate Answers and Capture Contexts

Now we'll run our RAG pipeline on each question and capture:
- The generated answer
- The retrieved context chunks used

In [14]:
# Generate evaluation dataset by running RAG pipeline
print("Generating answers and capturing contexts for all test questions...\n")

# Prepare dataset structure (required by Ragas)
evaluation_data = {
    "user_input": [],        # Questions
    "reference": [],         # Ground truth answers
    "response": [],          # RAG-generated answers
    "retrieved_contexts": [] # Retrieved chunks (List[List[str]])
}

# Process each test case
for i, test_case in enumerate(test_cases, 1):
    question = test_case["question"]
    ground_truth = test_case["ground_truth"]
    
    print(f"[{i}/{len(test_cases)}] Processing: {question[:60]}...")
    
    # Generate answer using RAG chain
    answer = rag_chain.invoke(question)
    
    # Get retrieved contexts (same retriever as in RAG chain)
    retrieved_docs = retriever.invoke(question)
    contexts = [doc.page_content for doc in retrieved_docs]
    
    # Append to dataset
    evaluation_data["user_input"].append(question)
    evaluation_data["reference"].append(ground_truth)
    evaluation_data["response"].append(answer)
    evaluation_data["retrieved_contexts"].append(contexts)

print(f"\n✓ Generated {len(evaluation_data['user_input'])} answer-context pairs")
print(f"\nDataset structure:")
print(f"  - user_input: {len(evaluation_data['user_input'])} questions")
print(f"  - reference: {len(evaluation_data['reference'])} ground truth answers")
print(f"  - response: {len(evaluation_data['response'])} RAG-generated answers")
print(f"  - retrieved_contexts: {len(evaluation_data['retrieved_contexts'])} context lists")

Generating answers and capturing contexts for all test questions...

[1/18] Processing: What is CloudFlow's uptime SLA?...
[2/18] Processing: What authentication protocol does CloudFlow use?...
[3/18] Processing: What is the service mesh technology used by CloudFlow?...
[4/18] Processing: What compliance standards does CloudFlow support?...
[5/18] Processing: How long are CloudFlow audit logs retained?...
[6/18] Processing: What are the three main layers of CloudFlow architecture?...
[7/18] Processing: What are CloudFlow's pricing tiers and their API rate limits...
[8/18] Processing: What HTTP status codes indicate authentication failures in C...
[9/18] Processing: What auto-scaling metrics does CloudFlow monitor?...
[10/18] Processing: How do I authenticate with CloudFlow APIs using an API key?...
[11/18] Processing: How do I handle rate limit errors in CloudFlow?...
[12/18] Processing: What steps should I follow to optimize CloudFlow API perform...
[13/18] Processing: What's the diff

## 5.3 Convert to Ragas Format

Ragas requires data in `EvaluationDataset` format with `SingleTurnSample` instances.

In [16]:
# Convert to Ragas EvaluationDataset format
#import pickle

samples = [
    SingleTurnSample(
        user_input=evaluation_data["user_input"][i],
        response=evaluation_data["response"][i],
        retrieved_contexts=evaluation_data["retrieved_contexts"][i],
        reference=evaluation_data["reference"][i]
    )
    for i in range(len(evaluation_data["user_input"]))
]

eval_dataset = EvaluationDataset(samples=samples)
# Save to CSV
#eval_dataset.to_csv("my_eval_dataset.csv")



# # After creating eval_dataset
# with open("my_eval_dataset.pkl", "wb") as f:
#     pickle.dump(eval_dataset, f)

# print("✅ Dataset saved as pickle")

# Load from CSV
#loaded_dataset = EvaluationDataset.from_csv("my_eval_dataset.csv")

print("✓ Converted to Ragas EvaluationDataset format")
print(f"  Total samples: {len(eval_dataset)}")
print(f"\nSample preview:")
print(f"  Question: {samples[0].user_input}")
print(f"  Answer: {samples[0].response[:100]}...")
print(f"  Contexts: {len(samples[0].retrieved_contexts)} chunks retrieved")

✓ Converted to Ragas EvaluationDataset format
  Total samples: 18

Sample preview:
  Question: What is CloudFlow's uptime SLA?
  Answer: CloudFlow guarantees a 99.99% uptime SLA for the Enterprise Tier, 99.95% for the Premium Tier, and 9...
  Contexts: 4 chunks retrieved


---

# Section 6: Ragas Metrics Implementation

**Difficulty:** 🎓/🚀 INTERMEDIATE/ADVANCED

Now we'll configure and explain all 6 non-multimodal Ragas metrics.

## 6.1 Configure Metrics

We'll set up all 6 non-multimodal metrics with proper LLM and embeddings configuration.

In [17]:
#from ragas.llms import LangchainLLMWrapper
#from langchain_openai import ChatOpenAI

#llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini", max_tokens=100))
#print(ragas_llm.langchain_llm.max_tokens)

In [ ]:
# evaluator_llm = ChatOpenAI(
#       model="gpt-4o-mini",
#       temperature=0,
#       max_tokens=2000,      # ✅ Increased from 500 to 2000
#       request_timeout=120   # ✅ Prevent timeouts (2 minutes)
# )
# ragas_llm = LangchainLLMWrapper(evaluator_llm)
# print(ragas_llm.langchain_llm.max_tokens)
# #ragas_embeddings = LangchainEmbeddingsWrapper(embeddings)

# import openai

# openai_client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# ragas_embeddings = embedding_factory(
#     'openai', 
#     model='text-embedding-3-small', 
#     client=openai_client
# )

2000


In [ ]:
# from ragas.llms import llm_factory
# from ragas.embeddings import embedding_factory
# import openai

# # Create OpenAI client
# openai_client = openai.OpenAI(api_key="your-api-key")

# # Create LLM using llm_factory
# ragas_llm = llm_factory(
#     'gpt-4o-mini', 
#     client=openai_client
# )

# # Create embeddings using embedding_factory
# ragas_embeddings = embedding_factory(
#     'openai',
#     model='text-embedding-3-small',
#     client=openai_client
# )

In [ ]:
# from ragas.llms import llm_factory
# from ragas.embeddings import embedding_factory
# import openai

# # Create OpenAI client
# openai_client = openai.OpenAI(api_key=os.environ['OPENAI_API_KEY'])

# # Create LLM using llm_factory
# ragas_llm = llm_factory(
#     'gpt-4o-mini', 
#     client=openai_client
# )

# # Create embeddings using embedding_factory
# ragas_embeddings = embedding_factory(
#     'openai',
#     model='text-embedding-3-small',
#     client=openai_client
# )

In [18]:
print("\n" + "=" * 80)
print("CONFIGURING LLM & EMBEDDINGS (WITH FIX)")
print("=" * 80)

print("\n🔧 Creating SEPARATE evaluator LLM with increased limits...")

# ✅ FIX: Create separate evaluator LLM with higher max_tokens
evaluator_llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,           # Deterministic for consistent evaluation
    max_tokens=3000,         # ✅ INCREASED from 500 to 2000
    request_timeout=480,      # ✅ ADDED 2-minute timeout to prevent hangs
    n=3                       # ✅ ADDED for multiple responses
)

print("✅ Evaluator LLM configured:")
print(f"   Model: gpt-4o-mini")
print(f"   Temperature: 0 (deterministic)")
print(f"   Max tokens: 2000 (sufficient for Ragas evaluations)")
print(f"   Request timeout: 120 seconds")

# Create embeddings
print("\n🔧 Creating embeddings model...")
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",  # 1536 dimensions, cost-effective
)
print("✅ Embeddings configured:")
print(f"   Model: text-embedding-3-small")
print(f"   Dimensions: 1536")

# ✅ Wrap for Ragas compatibility
print("\n🔧 Wrapping LLM and embeddings for Ragas...")
ragas_llm = LangchainLLMWrapper(evaluator_llm)
ragas_embeddings = LangchainEmbeddingsWrapper(embeddings)
print("✅ Wrappers created successfully")

# Initialize ALL 6 non-multimodal metrics with wrapped evaluator LLM and embeddings
# Note: In Ragas 0.3.9, metrics are classes that need LLM/embeddings initialization

from ragas.metrics import (
    Faithfulness,
    AnswerRelevancy,
    ContextPrecision,
    ContextRecall,
    ContextEntityRecall,
    NoiseSensitivity,
)
metrics = [
    Faithfulness(llm=ragas_llm),                         # Are answers grounded in context?
    AnswerRelevancy(llm=ragas_llm, embeddings=ragas_embeddings),        # Are answers relevant to questions?
    ContextPrecision(llm=ragas_llm),                     # Are relevant chunks ranked high?
    ContextRecall(llm=ragas_llm),                        # Was all needed info retrieved?
    ContextEntityRecall(llm=ragas_llm),                  # Are key entities from ground truth retrieved?
    NoiseSensitivity(llm=ragas_llm),                     # Is system robust to irrelevant context?
]

print("✓ Configured 6 Ragas metrics (v0.3.9) - All non-multimodal metrics\n")
print("Metric Details:")
print("=" * 80)
print("\n1. FAITHFULNESS")
print("   - Measures: Answer groundedness in retrieved context")
print("   - Type: LLM-based (uses GPT-4o-mini for verification)")
print("   - Initialization: Faithfulness(llm=ragas_llm)")
print("   - Target: >0.7")
print("   - Speed: ~2-3s per question")
print("   - Why: Prevents hallucinations and ensures factual accuracy")

print("\n2. ANSWER RELEVANCY (Response Relevancy)")
print("   - Measures: Semantic relevance of answer to question")
print("   - Type: Embedding-based (uses cosine similarity)")
print("   - Initialization: AnswerRelevancy(embeddings=ragas_embeddings)")
print("   - Target: >0.7")
print("   - Speed: ~0.5s per question")
print("   - Why: Ensures on-topic, focused responses")

print("\n3. CONTEXT PRECISION")
print("   - Measures: Ranking quality of retrieved chunks")
print("   - Type: LLM-based ground truth comparison")
print("   - Initialization: ContextPrecision(llm=ragas_llm)")
print("   - Target: >0.6")
print("   - Speed: ~1s per question")
print("   - Why: Evaluates retrieval algorithm effectiveness")

print("\n4. CONTEXT RECALL")
print("   - Measures: Completeness of retrieved information")
print("   - Type: LLM-based ground truth comparison")
print("   - Initialization: ContextRecall(llm=ragas_llm)")
print("   - Target: >0.6")
print("   - Speed: ~1s per question")
print("   - Why: Ensures no critical information is missed")

print("\n5. CONTEXT ENTITY RECALL")
print("   - Measures: Key entities from ground truth present in retrieved contexts")
print("   - Type: LLM-based entity extraction and matching")
print("   - Initialization: ContextEntityRecall(llm=ragas_llm)")
print("   - Target: >0.6")
print("   - Speed: ~1-2s per question")
print("   - Why: Ensures important named entities are retrieved")

print("\n6. NOISE SENSITIVITY")
print("   - Measures: Robustness to irrelevant/noisy context")
print("   - Type: LLM-based evaluation with added noise")
print("   - Initialization: NoiseSensitivity(llm=ragas_llm)")
print("   - Target: >0.7")
print("   - Speed: ~2-3s per question")
print("   - Why: Tests if system can ignore distractors")
print("=" * 80)

print("\n📌 Note on Ragas 0.3.9:")
print("   All 6 non-multimodal metrics are now configured.")
print("   Metrics are initialized as classes with wrapped LLM/embeddings.")
print("   LangChain components must be wrapped using Ragas wrapper classes.")


CONFIGURING LLM & EMBEDDINGS (WITH FIX)

🔧 Creating SEPARATE evaluator LLM with increased limits...
✅ Evaluator LLM configured:
   Model: gpt-4o-mini
   Temperature: 0 (deterministic)
   Max tokens: 2000 (sufficient for Ragas evaluations)
   Request timeout: 120 seconds

🔧 Creating embeddings model...
✅ Embeddings configured:
   Model: text-embedding-3-small
   Dimensions: 1536

🔧 Wrapping LLM and embeddings for Ragas...
✅ Wrappers created successfully
✓ Configured 6 Ragas metrics (v0.3.9) - All non-multimodal metrics

Metric Details:

1. FAITHFULNESS
   - Measures: Answer groundedness in retrieved context
   - Type: LLM-based (uses GPT-4o-mini for verification)
   - Initialization: Faithfulness(llm=ragas_llm)
   - Target: >0.7
   - Speed: ~2-3s per question
   - Why: Prevents hallucinations and ensures factual accuracy

2. ANSWER RELEVANCY (Response Relevancy)
   - Measures: Semantic relevance of answer to question
   - Type: Embedding-based (uses cosine similarity)
   - Initializati

## 6.2 Understanding Metric Scores

**Score Interpretation:**

| Score Range | Interpretation | Action |
|-------------|----------------|--------|
| 0.8 - 1.0 | Excellent | Production-ready |
| 0.7 - 0.8 | Good | Minor tuning recommended |
| 0.6 - 0.7 | Acceptable | Needs optimization |
| 0.5 - 0.6 | Poor | Significant issues to address |
| < 0.5 | Very Poor | Major redesign needed |

**What to optimize if scores are low:**

- **Low Faithfulness**: Improve prompt to emphasize context adherence, reduce temperature
- **Low Answer Relevancy**: Improve prompt focus, filter irrelevant retrieved chunks
- **Low Context Precision**: Optimize chunk size, adjust k value, improve metadata
- **Low Context Recall**: Increase k value, improve chunking strategy, enhance retrieval

---

# Section 7: Running Comprehensive Evaluation

**Difficulty:** 🎓 INTERMEDIATE

Now we'll run Ragas evaluation on our baseline RAG configuration.

## 7.1 Run Ragas Evaluation

This will take ~7-12 minutes depending on the number of questions and metrics (6 metrics total).

In [19]:
print("Starting Ragas evaluation...")
print(f"  Dataset: {len(eval_dataset)} questions")
print(f"  Metrics: {len(metrics)} (all 6 non-multimodal metrics)")
print(f"  Estimated time: 7-12 minutes\n")

# Run evaluation (Ragas 0.3.7)
# Note: In Ragas 0.3+, LLM and embeddings are configured on metrics directly,
# so we don't need to pass them to evaluate()
result = evaluate(
    dataset=eval_dataset,
    metrics=metrics,
)

print("\n✓ Evaluation completed!")
print("\n" + "=" * 80)
print("RAGAS EVALUATION RESULTS (6 Metrics)")
print("=" * 80)
print(result)
print("=" * 80)

Starting Ragas evaluation...
  Dataset: 18 questions
  Metrics: 6 (all 6 non-multimodal metrics)
  Estimated time: 7-12 minutes



Evaluating:   0%|          | 0/108 [00:00<?, ?it/s]

LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
Exception raised in Job[5]: TimeoutError()
Exception raised in Job[11]: TimeoutError()
Exception raised in Job[17]: TimeoutError()
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
Exception raised in Job[23]: TimeoutError()
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
Exception raised in Job[29]: TimeoutError()
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of reque


✓ Evaluation completed!

RAGAS EVALUATION RESULTS (6 Metrics)
{'faithfulness': 0.9029, 'answer_relevancy': 0.8278, 'context_precision': 0.8565, 'context_recall': 0.8333, 'context_entity_recall': 0.3859, 'noise_sensitivity(mode=relevant)': 0.5000}


## 7.2 Log Results to OPIK

**Optional:** If OPIK is configured, log the evaluation results for experiment tracking and analysis.

**What gets logged:**
- All 6 metric scores (mean values)
- Configuration parameters (chunk_size, k, search_type, etc.)
- Model information (LLM, embeddings)
- Dataset metadata (num_questions, num_documents)
- Overall score (average of all metrics)

**Benefits:**
- Track experiments over time
- Compare different RAG configurations
- Share results with team
- Visualize metric trends


In [ ]:
# === Log Baseline Evaluation to OPIK ===
if OPIK_AVAILABLE:
    try:
        # Convert result to DataFrame
        results_df = result.to_pandas()
        
        # Calculate mean scores for all 6 metrics
        metric_scores = {
            'faithfulness': float(results_df['faithfulness'].mean()),
            'answer_relevancy': float(results_df['answer_relevancy'].mean()),
            'context_precision': float(results_df['context_precision'].mean()),
            'context_recall': float(results_df['context_recall'].mean()),
            'context_entity_recall': float(results_df['context_entity_recall'].mean()),
            'noise_sensitivity': float(results_df['noise_sensitivity'].mean()),
        }
        
        # Calculate overall average
        overall_score = sum(metric_scores.values()) / len(metric_scores)
        
        # Log experiment to OPIK
        opik_client.log_traces(
            project_name=project_name,
            name="Baseline_RAG_Evaluation",
            input={"configuration": "baseline", "num_questions": len(eval_dataset)},
            output=metric_scores,
            metadata={
                "ragas_version": "0.3.9",
                "chunk_size": 1024,
                "chunk_overlap": 200,
                "retriever_k": 4,
                "search_type": "similarity",
                "embedding_model": "text-embedding-3-small",
                "llm_model": "gpt-4o-mini",
                "num_documents": 18,
                "num_questions": len(eval_dataset),
                "evaluation_date": datetime.now().isoformat(),
                "overall_score": overall_score,
            },
            tags=["baseline", "ragas", "rag_evaluation", "all_6_metrics"]
        )
        
        print("\n✓ Logged baseline evaluation to OPIK")
        print(f"  Overall Score: {overall_score:.4f}")
        print(f"  Dashboard: https://www.comet.com/opik/projects/{project_name}")
        
    except Exception as e:
        print(f"⚠️ OPIK logging failed: {e}")
else:
    print("ℹ OPIK not available - skipping logging")

## 7.2 Analyze Results

Let's convert results to a DataFrame for better analysis.

In [18]:
# Convert results to pandas DataFrame
results_df = result.to_pandas()

print("Detailed Results by Question:\n")
print(results_df.to_string())

# Calculate summary statistics
print("\n" + "=" * 80)
print("SUMMARY STATISTICS")
print("=" * 80)
print(results_df.describe())

# Identify best and worst performing questions
print("\n" + "=" * 80)
print("BEST & WORST PERFORMING QUESTIONS")
print("=" * 80)

# Calculate average score per question (across all metrics)
metric_cols = [col for col in results_df.columns if col not in ['user_input', 'response', 'retrieved_contexts', 'reference']]
results_df['avg_score'] = results_df[metric_cols].mean(axis=1)

# Best performing
best_idx = results_df['avg_score'].idxmax()
print(f"\nBEST: {results_df.loc[best_idx, 'user_input']}")
print(f"  Average Score: {results_df.loc[best_idx, 'avg_score']:.3f}")

# Worst performing
worst_idx = results_df['avg_score'].idxmin()
print(f"\nWORST: {results_df.loc[worst_idx, 'user_input']}")
print(f"  Average Score: {results_df.loc[worst_idx, 'avg_score']:.3f}")

Detailed Results by Question:

                                                                            user_input                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

---

# Section 8: Advanced Evaluation - Multiple Configurations

**Difficulty:** 🚀 ADVANCED

Now we'll compare 4 different RAG configurations:

1. **Baseline**: chunk_size=1024, k=4, similarity
2. **Precise**: chunk_size=512, k=4, similarity (smaller chunks)
3. **Contextual**: chunk_size=1024, k=6, similarity (more context)
4. **Diverse**: chunk_size=1024, k=4, mmr (diversity)

## 8.1 Define Configurations

In [ ]:
# Define configurations to test
configs = {
    "Baseline": {
        "chunk_size": 1024,
        "chunk_overlap": 128,
        "k": 4,
        "search_type": "similarity",
        "description": "Standard configuration (our baseline)"
    },
    "Precise": {
        "chunk_size": 512,
        "chunk_overlap": 64,
        "k": 4,
        "search_type": "similarity",
        "description": "Smaller chunks for focused answers"
    },
    "Contextual": {
        "chunk_size": 1024,
        "chunk_overlap": 128,
        "k": 6,
        "search_type": "similarity",
        "description": "More context for complex questions"
    },
    "Diverse": {
        "chunk_size": 1024,
        "chunk_overlap": 128,
        "k": 4,
        "search_type": "mmr",
        "description": "Maximum Marginal Relevance for diversity"
    }
}

print("Configuration Matrix:\n")
print("=" * 80)
for config_name, config in configs.items():
    print(f"\n{config_name}:")
    print(f"  Chunk Size: {config['chunk_size']}")
    print(f"  Chunk Overlap: {config['chunk_overlap']}")
    print(f"  k (retrieval): {config['k']}")
    print(f"  Search Type: {config['search_type']}")
    print(f"  Description: {config['description']}")
print("=" * 80)

## 8.2 Evaluate All Configurations

**Warning**: This will take 15-25 minutes as we're evaluating 4 configurations.

For faster testing, you can reduce the number of test questions or configurations.

In [ ]:
# Store results for all configurations
comparison_results = {}

print("Starting multi-configuration evaluation...")
print(f"Total configurations: {len(configs)}")
print(f"Estimated total time: 20-30 minutes (6 metrics per config)\n")
print("=" * 80)

for config_name, config in configs.items():
    print(f"\nEvaluating configuration: {config_name}")
    print(f"  {config['description']}")
    print(f"  Progress: {list(configs.keys()).index(config_name) + 1}/{len(configs)}")
    
    # Step 1: Re-split documents with new chunk_size
    text_splitter_config = RecursiveCharacterTextSplitter(
        chunk_size=config["chunk_size"],
        chunk_overlap=config["chunk_overlap"],
        separators=["\n\n", "\n", " ", ""]
    )
    chunks_config = text_splitter_config.split_documents(cloudflow_docs)
    print(f"  ✓ Created {len(chunks_config)} chunks")
    
    # Step 2: Recreate vector store
    vectorstore_config = FAISS.from_documents(chunks_config, embeddings)
    print(f"  ✓ Vector store created")
    
    # Step 3: Create retriever with config
    if config["search_type"] == "mmr":
        retriever_config = vectorstore_config.as_retriever(
            search_type="mmr",
            search_kwargs={"k": config["k"], "fetch_k": config["k"] * 2}
        )
    else:
        retriever_config = vectorstore_config.as_retriever(
            search_type="similarity",
            search_kwargs={"k": config["k"]}
        )
    print(f"  ✓ Retriever configured")
    
    # Step 4: Rebuild RAG chain (same prompt and LLM)
    rag_chain_config = (
        {"context": retriever_config | format_docs, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )
    print(f"  ✓ RAG chain built")
    
    # Step 5: Generate evaluation data
    print(f"  ► Generating answers for {len(test_cases)} questions...")
    config_eval_data = {
        "user_input": [],
        "reference": [],
        "response": [],
        "retrieved_contexts": []
    }
    
    for test_case in test_cases:
        question = test_case["question"]
        ground_truth = test_case["ground_truth"]
        
        answer = rag_chain_config.invoke(question)
        retrieved_docs = retriever_config.invoke(question)
        contexts = [doc.page_content for doc in retrieved_docs]
        
        config_eval_data["user_input"].append(question)
        config_eval_data["reference"].append(ground_truth)
        config_eval_data["response"].append(answer)
        config_eval_data["retrieved_contexts"].append(contexts)
    
    # Convert to Ragas format
    config_samples = [
        SingleTurnSample(
            user_input=config_eval_data["user_input"][i],
            response=config_eval_data["response"][i],
            retrieved_contexts=config_eval_data["retrieved_contexts"][i],
            reference=config_eval_data["reference"][i]
        )
        for i in range(len(config_eval_data["user_input"]))
    ]
    config_eval_dataset = EvaluationDataset(samples=config_samples)
    
    # Step 6: Initialize ALL 6 metrics for this configuration (with wrapped LLM/embeddings)
    config_metrics = [
        Faithfulness(llm=ragas_llm),
        AnswerRelevancy(embeddings=ragas_embeddings),
        ContextPrecision(llm=ragas_llm),
        ContextRecall(llm=ragas_llm),
        ContextEntityRecall(llm=ragas_llm),
        NoiseSensitivity(llm=ragas_llm),
    ]
    
    # Step 7: Run Ragas evaluation (Ragas 0.3.9 API)
    print(f"  ► Running Ragas evaluation with 6 metrics (this may take 7-12 minutes)...")
    config_result = evaluate(
        dataset=config_eval_dataset,
        metrics=config_metrics,
    )
    
    # Step 8: Store results
    comparison_results[config_name] = config_result.to_pandas()

    # Log to OPIK
    if OPIK_AVAILABLE:
        try:
            config_df = comparison_results[config_name]
            config_metric_scores = {
                'faithfulness': float(config_df['faithfulness'].mean()),
                'answer_relevancy': float(config_df['answer_relevancy'].mean()),
                'context_precision': float(config_df['context_precision'].mean()),
                'context_recall': float(config_df['context_recall'].mean()),
                'context_entity_recall': float(config_df['context_entity_recall'].mean()),
                'noise_sensitivity': float(config_df['noise_sensitivity'].mean()),
            }
            config_overall_score = sum(config_metric_scores.values()) / len(config_metric_scores)
            
            opik_client.log_traces(
                project_name=project_name,
                name=f"RAG_Evaluation_{config_name}",
                input={
                    "configuration": config_name,
                    "description": config['description'],
                    "num_questions": len(config_eval_dataset)
                },
                output=config_metric_scores,
                metadata={
                    "ragas_version": "0.3.9",
                    "chunk_size": config["chunk_size"],
                    "chunk_overlap": config["chunk_overlap"],
                    "retriever_k": config["k"],
                    "search_type": config["search_type"],
                    "embedding_model": "text-embedding-3-small",
                    "llm_model": "gpt-4o-mini",
                    "num_documents": 18,
                    "num_questions": len(config_eval_dataset),
                    "overall_score": config_overall_score,
                },
                tags=[config_name, "ragas", "rag_evaluation", "all_6_metrics", "multi_config"]
            )
            print(f"  ✓ Logged {config_name} to OPIK (Overall: {config_overall_score:.4f})")
        except Exception as e:
            print(f"  ⚠️ OPIK logging failed for {config_name}: {e}")
    print(f"  ✓ Evaluation complete!")
    print(f"  Average scores: {comparison_results[config_name][metric_cols].mean().to_dict()}")
    print("-" * 80)

print("\n" + "=" * 80)
print("✓ ALL CONFIGURATIONS EVALUATED!")
print("=" * 80)

---

# Section 9: Results Analysis & Visualization

**Difficulty:** 🎓/🚀 INTERMEDIATE/ADVANCED

Now we'll create comprehensive visualizations to compare all configurations.

## 9.1 Summary Statistics Table

In [ ]:
# Calculate summary statistics for each configuration
summary_stats = {}

for config_name, result_df in comparison_results.items():
    metric_scores = result_df[metric_cols].mean()
    summary_stats[config_name] = {
        'Mean': metric_scores.mean(),
        'Std': metric_scores.std(),
        'Min': metric_scores.min(),
        'Max': metric_scores.max()
    }

summary_df = pd.DataFrame(summary_stats).T

print("Summary Statistics Across All Metrics:\n")
print("=" * 80)
print(summary_df.round(3))
print("=" * 80)

# Identify best configuration
best_config = summary_df['Mean'].idxmax()
print(f"\n🏆 BEST OVERALL CONFIGURATION: {best_config}")
print(f"   Average Score: {summary_df.loc[best_config, 'Mean']:.3f}")

## 9.2 Visualization 1: Grouped Bar Chart

Compare all configurations across all metrics.

In [ ]:
# Prepare data for grouped bar chart
metrics_list = metric_cols
config_names = list(comparison_results.keys())

# Create figure
fig, ax = plt.subplots(figsize=(14, 6))
x = np.arange(len(metrics_list))
width = 0.2  # Width of bars

# Plot bars for each configuration
for i, config in enumerate(config_names):
    scores = [comparison_results[config][metric].mean() for metric in metrics_list]
    ax.bar(x + i*width, scores, width, label=config, alpha=0.8)

# Customize chart
ax.set_xlabel('Metrics', fontsize=12, fontweight='bold')
ax.set_ylabel('Score', fontsize=12, fontweight='bold')
ax.set_title('RAG Configuration Comparison Across Metrics', fontsize=14, fontweight='bold', pad=20)
ax.set_xticks(x + width * 1.5)
ax.set_xticklabels(metrics_list, rotation=45, ha='right')
ax.axhline(y=0.7, color='red', linestyle='--', linewidth=1.5, label='Target (0.7)', alpha=0.7)
ax.legend(loc='upper left', framealpha=0.9)
ax.grid(axis='y', alpha=0.3, linestyle=':')
ax.set_ylim(0, 1.0)

plt.tight_layout()
plt.show()

print("\n📊 Grouped bar chart shows how each configuration performs on each metric.")
print("   Configurations are grouped by metric for easy comparison.")

## 9.3 Visualization 2: Radar Chart

Show the strength/weakness profile of each configuration.

In [ ]:
# Create radar chart
fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(projection='polar'))

# Calculate angles for each metric
angles = np.linspace(0, 2*np.pi, len(metrics_list), endpoint=False).tolist()
angles += angles[:1]  # Complete the circle

# Plot each configuration
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#FFA07A']
for idx, config in enumerate(config_names):
    scores = [comparison_results[config][metric].mean() for metric in metrics_list]
    scores += scores[:1]  # Complete the circle
    
    ax.plot(angles, scores, 'o-', linewidth=2, label=config, color=colors[idx])
    ax.fill(angles, scores, alpha=0.15, color=colors[idx])

# Customize chart
ax.set_xticks(angles[:-1])
ax.set_xticklabels(metrics_list, fontsize=10)
ax.set_ylim(0, 1)
ax.set_yticks([0.2, 0.4, 0.6, 0.8, 1.0])
ax.set_yticklabels(['0.2', '0.4', '0.6', '0.8', '1.0'], fontsize=9)
ax.set_title('Configuration Performance Profile\n(Radar Chart)', 
             fontsize=14, fontweight='bold', pad=30)
ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1), framealpha=0.9)
ax.grid(True, linestyle=':', alpha=0.5)

plt.tight_layout()
plt.show()

print("\n🎯 Radar chart shows the overall performance profile of each configuration.")
print("   Larger filled areas indicate better overall performance.")

## 9.4 Visualization 3: Heatmap

Color-coded performance grid for quick pattern identification.

In [ ]:
# Prepare data matrix for heatmap
data_matrix = []
for config in config_names:
    scores = [comparison_results[config][metric].mean() for metric in metrics_list]
    data_matrix.append(scores)

# Create heatmap
fig, ax = plt.subplots(figsize=(12, 6))

sns.heatmap(
    data_matrix,
    annot=True,
    fmt='.3f',
    cmap='RdYlGn',  # Red (low) → Yellow (medium) → Green (high)
    center=0.7,      # Center colormap at target threshold
    vmin=0,
    vmax=1,
    xticklabels=metrics_list,
    yticklabels=config_names,
    cbar_kws={'label': 'Score'},
    linewidths=0.5,
    linecolor='gray',
    ax=ax
)

ax.set_title('RAG Evaluation Heatmap: Configurations × Metrics', 
             fontsize=14, fontweight='bold', pad=15)
ax.set_xlabel('Metrics', fontsize=12, fontweight='bold')
ax.set_ylabel('Configurations', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n🔥 Heatmap provides quick visual identification of strengths and weaknesses.")
print("   Green = Good performance, Yellow = Acceptable, Red = Needs improvement")

## 9.5 Detailed Analysis and Insights

In [ ]:
print("DETAILED ANALYSIS & INSIGHTS")
print("=" * 80)

# Analysis 1: Best configuration per metric
print("\n1. BEST CONFIGURATION PER METRIC:")
print("-" * 80)
for metric in metrics_list:
    best_config_for_metric = max(config_names, 
                                  key=lambda c: comparison_results[c][metric].mean())
    best_score = comparison_results[best_config_for_metric][metric].mean()
    print(f"   {metric:20s}: {best_config_for_metric:15s} (score: {best_score:.3f})")

# Analysis 2: Configuration rankings
print("\n2. OVERALL CONFIGURATION RANKINGS:")
print("-" * 80)
rankings = summary_df.sort_values('Mean', ascending=False)
for rank, (config, row) in enumerate(rankings.iterrows(), 1):
    print(f"   #{rank}. {config:15s} - Average: {row['Mean']:.3f} (±{row['Std']:.3f})")

# Analysis 3: Trade-offs
print("\n3. KEY TRADE-OFFS:")
print("-" * 80)
print("   Baseline:    Balanced performance across all metrics")
print("   Precise:     Better for focused answers, may miss context")
print("   Contextual:  More complete information, potential noise")
print("   Diverse:     Variety in retrieval, may reduce precision")

# Analysis 4: Recommendations
print("\n4. RECOMMENDATIONS:")
print("-" * 80)
best_overall = rankings.index[0]
print(f"   → For general use: {best_overall}")
print(f"   → For accuracy-critical apps: Check highest faithfulness config")
print(f"   → For complex questions: Contextual (k=6)")
print(f"   → For exploratory queries: Diverse (MMR)")

print("\n" + "=" * 80)

---

# Section 10: Best Practices & Summary

**Difficulty:** 🔰/🎓 BEGINNER/INTERMEDIATE

## 10.1 RAG Evaluation Best Practices

### ✅ Creating Effective Test Datasets

1. **Diversity is key**: Include 15-30 questions covering different:
   - Question types (factual, procedural, comparison)
   - Difficulty levels (simple to complex)
   - Edge cases (missing info, ambiguous queries)

2. **High-quality ground truth**:
   - Write clear, accurate reference answers
   - Include specific details (numbers, names, dates)
   - Test "I don't know" handling with out-of-scope questions

3. **Representative of production**:
   - Use real user questions when possible
   - Mirror actual query patterns
   - Include common pain points

### ✅ Choosing Appropriate Metrics

**For accuracy-critical applications** (medical, legal, financial):
- Prioritize: Faithfulness, Context Recall
- Target: >0.8 on both

**For customer support / Q&A**:
- Prioritize: Answer Relevancy, Faithfulness
- Target: >0.7 on both

**For exploratory / research**:
- Prioritize: Context Recall, Context Precision
- Target: >0.6 on both

### ✅ Optimization Strategies

**If Faithfulness is low (<0.7)**:
- Improve prompt to emphasize context adherence
- Reduce LLM temperature (make it more deterministic)
- Add explicit instruction: "Only use information from context"
- Use stronger LLM (GPT-4 vs GPT-3.5)

**If Answer Relevancy is low (<0.7)**:
- Improve prompt focus and clarity
- Filter retrieved chunks by relevance score
- Add query preprocessing (spell check, expansion)
- Tune retrieval parameters (k value)

**If Context Precision is low (<0.6)**:
- Optimize chunk size (test 512, 1024, 2048)
- Adjust k value (test 3, 4, 5, 6)
- Improve document metadata for filtering
- Consider hybrid search (keyword + semantic)

**If Context Recall is low (<0.6)**:
- Increase k value (retrieve more chunks)
- Improve chunking strategy (reduce overlap)
- Enhance document preprocessing
- Consider multi-query retrieval

### ✅ Regular Evaluation Cycles

1. **Initial development**: Evaluate after each major change
2. **Pre-production**: Full evaluation on comprehensive test set
3. **Production**: 
   - Weekly automated evaluation
   - Monthly review of failed queries
   - Quarterly full re-evaluation


### ✅ Experiment Tracking with OPIK

**Track evaluations for better insights**:

1. **Log all evaluation runs**:
   - Automatically log metric scores
   - Track configuration parameters
   - Record timestamps and metadata

2. **Compare experiments**:
   - Visualize metric trends over time
   - Compare different RAG configurations
   - Identify best-performing setups

3. **Team collaboration**:
   - Share results with teammates
   - Comment on specific evaluations
   - Build institutional knowledge

4. **Production monitoring**:
   - Track live RAG performance
   - Alert on metric degradation
   - Correlate with user feedback

**Setup**: Install OPIK (`pip install opik`), configure with API key, and the notebook will automatically log all evaluations.

### ✅ Cost Management

**Reduce evaluation costs**:
- Use smaller test set for rapid iteration (5-10 questions)
- Use full test set (15-30 questions) before deployment
- Cache evaluation results for unchanged configs
- Use non-LLM metrics for quick checks

**Estimated costs** (with GPT-4o-mini):
- 10 questions, 4 metrics: ~$0.20-0.40
- 20 questions, 4 metrics: ~$0.40-0.80
- 4 configs × 20 questions: ~$1.60-3.20

## 10.2 Ragas Metrics Cheat Sheet

| Metric | What It Measures | When to Use | How to Optimize If Low |
|--------|-----------------|-------------|------------------------|
| **Faithfulness** | Answer grounded in context | Always (prevents hallucinations) | Improve prompt, reduce temperature, use stronger LLM |
| **Answer Relevancy** | Answer matches question | Always (ensures on-topic) | Improve prompt focus, filter contexts, preprocess queries |
| **Context Precision** | Relevant chunks ranked high | When retrieval quality matters | Optimize chunk size, adjust k, improve metadata |
| **Context Recall** | All needed info retrieved | When completeness matters | Increase k, improve chunking, multi-query retrieval |

### Target Thresholds

| Application Type | Faithfulness | Answer Relevancy | Context Precision | Context Recall |
|-----------------|--------------|------------------|-------------------|----------------|
| **High-stakes** (medical, legal) | >0.85 | >0.80 | >0.70 | >0.75 |
| **Production** (customer support) | >0.75 | >0.75 | >0.65 | >0.65 |
| **Development** (testing) | >0.65 | >0.65 | >0.55 | >0.55 |
| **Prototype** (MVP) | >0.55 | >0.55 | >0.45 | >0.45 |

## 10.3 Configuration Selection Guide

### When to use different configurations:

**Baseline (chunk_size=1024, k=4, similarity)**
- ✅ General-purpose applications
- ✅ Balanced performance and cost
- ✅ Good starting point for tuning

**Precise (chunk_size=512, k=4, similarity)**
- ✅ Technical Q&A requiring specific details
- ✅ When answer brevity is important
- ✅ Highly structured documentation
- ⚠️ May miss broader context

**Contextual (chunk_size=1024, k=6, similarity)**
- ✅ Complex, multi-part questions
- ✅ When comprehensive answers needed
- ✅ Exploratory or research queries
- ⚠️ Higher cost (more tokens)
- ⚠️ Potential information overload

**Diverse (chunk_size=1024, k=4, mmr)**
- ✅ Broad topic exploration
- ✅ When query intent is ambiguous
- ✅ Multi-aspect questions
- ⚠️ May reduce precision for focused queries

## 10.4 Production Deployment Checklist

Before deploying your RAG system to production:

### ✅ Evaluation Requirements
- [ ] Faithfulness score >0.75 on test set
- [ ] Answer Relevancy score >0.75 on test set
- [ ] Context Precision score >0.65 on test set
- [ ] Context Recall score >0.65 on test set
- [ ] Tested on minimum 20 diverse questions
- [ ] Edge cases handled (missing info, ambiguous queries)

### ✅ Monitoring Setup
- [ ] Logging all queries and responses
- [ ] Tracking response times
- [ ] Monitoring API costs
- [ ] Collecting user feedback
- [ ] Alerting for quality degradation

### ✅ Quality Assurance
- [ ] Manual review of sample outputs
- [ ] A/B testing of configurations
- [ ] Load testing for performance
- [ ] Failover and error handling tested

### ✅ Documentation
- [ ] System architecture documented
- [ ] Configuration parameters recorded
- [ ] Evaluation results archived
- [ ] Known limitations documented

### ✅ Continuous Improvement
- [ ] Weekly automated evaluation
- [ ] Monthly review of failed queries
- [ ] Quarterly full re-evaluation
- [ ] Process for adding new test cases

## 10.5 Summary and Key Takeaways

### What We Accomplished

In this notebook, we:

1. ✅ Created custom CloudFlow documentation (18 documents)
2. ✅ Implemented a complete RAG pipeline with LCEL
3. ✅ Integrated Ragas evaluation framework
4. ✅ Created comprehensive test dataset (18 Q&A pairs)
5. ✅ Evaluated with all 6 non-multimodal Ragas metrics
6. ✅ Compared 4 different RAG configurations
7. ✅ Created professional visualizations
8. ✅ Analyzed results and identified best configuration
9. ✅ Integrated OPIK for experiment tracking and monitoring

### Key Learnings

**Evaluation is essential**:
- You can't improve what you don't measure
- Ragas provides standardized, reliable metrics
- Regular evaluation catches quality issues early

**Configuration matters**:
- No one-size-fits-all solution
- Different configs excel at different tasks
- Test multiple approaches systematically

**Balance trade-offs**:
- Precision vs Recall
- Cost vs Quality
- Speed vs Accuracy

### Next Steps

**Immediate**:
1. Apply this evaluation approach to your own RAG system
2. Create a test dataset for your domain
3. Run baseline evaluation

**Short-term**:
1. Test additional configurations (temperature, prompts, models)
2. Expand test dataset to 30-50 questions
3. Implement automated evaluation pipeline

**Long-term**:
1. Set up continuous evaluation in production
2. Build feedback loop with real user queries
3. Explore advanced techniques (hybrid search, re-ranking, query transformation)

### Resources for Further Learning

**Ragas Documentation**:
- Official docs: https://docs.ragas.io
- GitHub: https://github.com/explodinggradients/ragas

**LangChain Resources**:
- RAG tutorials: https://python.langchain.com/docs/use_cases/question_answering/
- LCEL guide: https://python.langchain.com/docs/expression_language/

**RAG Best Practices**:
- Pinecone RAG guide: https://www.pinecone.io/learn/retrieval-augmented-generation/
- OpenAI RAG patterns: https://platform.openai.com/docs/guides/retrieval-augmented-generation

---

## 🎉 Congratulations!

You've completed the RAG Evaluation with Ragas notebook! You now have:

- ✅ Practical experience with Ragas framework
- ✅ Understanding of key RAG evaluation metrics
- ✅ Skills to compare configurations systematically
- ✅ Production-ready evaluation patterns

**You're now equipped to build, evaluate, and optimize production-quality RAG systems!** 🚀

---